In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,43208
2,Huelva,Confirmados PDIA 14 días,2254
3,Huelva,Tasa PDIA 14 días,"439,230664302278"
4,Huelva,Confirmados PDIA 7 días,904
5,Huelva,Tasa PDIA 7 dias,"176,15994699612213"
6,Huelva,Total Confirmados,43407
7,Huelva,Curados,36511
8,Huelva,Fallecidos,397


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  43208.0


/tmp/ipykernel_3664/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12070.0


/tmp/ipykernel_3664/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3664/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3664/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3664/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 543 personas en los últimos 7 días 

Un positivo PCR cada 216 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,43208.0,904.0,2254.0,513170.0,176.159947,439.230664,422.0
Huelva-Costa,25481.0,594.0,1455.0,289548.0,205.147333,502.507356,295.0
Huelva (capital),12070.0,265.0,667.0,143837.0,184.236323,463.719349,126.0
Condado-Campiña,13507.0,223.0,621.0,156231.0,142.737357,397.488335,89.0
Isla Cristina,2936.0,73.0,169.0,21393.0,341.233114,789.978030,42.0
Lepe,2859.0,58.0,168.0,27880.0,208.034433,602.582496,40.0
Cartaya,1762.0,63.0,132.0,20083.0,313.698153,657.272320,34.0
Ayamonte,1919.0,82.0,168.0,21104.0,388.551933,796.057619,33.0
Sierra de Huelva-Andévalo Central,3820.0,78.0,163.0,67391.0,115.742458,241.872060,32.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cañaveral de León,21.0,3.0,8.0,396.0,757.575758,2020.202020,0.0
Linares de la Sierra,9.0,5.0,5.0,262.0,1908.396947,1908.396947,4.0
Alájar,30.0,8.0,11.0,759.0,1054.018445,1449.275362,3.0
Corteconcepción,45.0,2.0,6.0,536.0,373.134328,1119.402985,1.0
Ayamonte,1919.0,82.0,168.0,21104.0,388.551933,796.057619,33.0
Isla Cristina,2936.0,73.0,169.0,21393.0,341.233114,789.978030,42.0
Campofrío,20.0,2.0,5.0,713.0,280.504909,701.262272,0.0
Villablanca,263.0,7.0,19.0,2885.0,242.634315,658.578856,5.0
Cartaya,1762.0,63.0,132.0,20083.0,313.698153,657.272320,34.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lucena del Puerto,343.0,1.0,9.0,3261.0,30.665440,275.988960,0.0,0.111111
Villarrasa,216.0,2.0,13.0,2211.0,90.456807,587.969245,1.0,0.153846
Manzanilla,136.0,1.0,6.0,2118.0,47.214353,283.286119,1.0,0.166667
Trigueros,523.0,10.0,51.0,7862.0,127.194098,648.689901,4.0,0.196078
Escacena del Campo,209.0,3.0,15.0,2287.0,131.176213,655.881067,2.0,0.200000
Bonares,441.0,6.0,28.0,6060.0,99.009901,462.046205,3.0,0.214286
Cortegana,283.0,4.0,17.0,4602.0,86.918731,369.404607,1.0,0.235294
Villalba del Alcor,493.0,1.0,4.0,3366.0,29.708853,118.835413,1.0,0.250000
Galaroza,54.0,1.0,4.0,1382.0,72.358900,289.435601,0.0,0.250000
